In [1]:
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
# Load the VGG16 model
vgg16_model = VGG16(weights='imagenet')

# Load the VGG19 model
vgg19_model = VGG19(weights='imagenet')


In [ ]:
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # Preprocess for VGG models
    return img_array

def predict_image_vgg16(img_path):
    img_array = load_and_preprocess_image(img_path)
    predictions = vgg16_model.predict(img_array)
    decoded_predictions = decode_predictions(predictions, top=3)  # Show top-3 predictions
    return decoded_predictions

def predict_image_vgg19(img_path):
    img_array = load_and_preprocess_image(img_path)
    predictions = vgg19_model.predict(img_array)
    decoded_predictions = decode_predictions(predictions, top=3)
    return decoded_predictions



In [ ]:
# Path to the image
img_path = 'germa.jpg'

# Predict with VGG16
vgg16_predictions = predict_image_vgg16(img_path)
print("VGG16 Predictions:")
for _, label, score in vgg16_predictions[0]:
    print(f"{label}: {score * 100:.2f}%")

# Predict with VGG19
vgg19_predictions = predict_image_vgg19(img_path)
print("\nVGG19 Predictions:")
for _, label, score in vgg19_predictions[0]:
    print(f"{label}: {score * 100:.2f}%")